In [24]:
from IPython.display import display, Math, Latex

import pandas as pd
import numpy as np
import numpy_financial as npf
import yfinance as yf
import matplotlib.pyplot as plt
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

#### Group Assignment
- Team Number: <font color="yellow"> 12 </font>
- Team Member Names: <font color="yellow"> Robin Zhang, Jessica Chen, Eric Wang </font>
- Team Strategy Chosen: <font color="red"> Risky </font>

In [29]:
start_date = "2019-11-24"
end_date = "2022-11-24"

In [30]:
#Call the CSV file and store it in a dataframe
tickers = pd.read_csv('Tickers_Example.csv')
length = len(tickers)

#get first value
colname = tickers.columns[0]
missing_data = pd.DataFrame({'AAPL': [colname]})

tickers = tickers.append(missing_data)

tickers.reset_index(inplace=True, drop=True)

#make list of tickers
#tickers is now AAPL because it does not work if tickers is called tickers
#I am not sure why
ticker_list = list(tickers.AAPL)

ticker_data = {}
new_lst = []
i=0

for t in ticker_list:
    firm = yf.Ticker(ticker_list[i])
    firm_hist = firm.history(start=start_date, end=end_date, interval='1mo')

    ticker_data[t] = firm_hist.Volume

    if ticker_data[t].mean() > 200000:
        #make a new lst that stores the values where mean >200 000
        #return that list
        new_lst.append(ticker_list[i])

    i = i + 1

ticker_list = new_lst 
for t in ticker_list:
    if "." in t:
        ticker_list.remove(t)
        
ticker_list

- AGN: No data found, symbol may be delisted
- CELG: No data found, symbol may be delisted
- PCLN: No data found for this date range, symbol may be delisted
- RTN: No data found, symbol may be delisted
- TWX: No data found for this date range, symbol may be delisted
['ABBV', 'ABT', 'ACN', 'AIG', 'AMZN', 'AXP', 'BA', 'BAC', 'BIIB', 'BK', 'BLK', 'BMY', 'C', 'CAT', 'CL', 'CMCSA', 'COF', 'COP', 'COST', 'CSCO', 'CVS', 'GM', 'GOOG', 'JPM', 'KMI', 'KO', 'LLY', 'LMT', 'MO', 'MON', 'MRK', 'MS', 'MSFT', 'NEE', 'NKE', 'ORCL', 'OXY', 'PEP', 'PFE', 'PG', 'PM', 'PYPL', 'QCOM', 'SBUX', 'SLB', 'SO', 'SPG', 'T', 'TGT', 'TXN', 'UNH', 'UNP', 'UPS', 'USB', 'AAPL']


In [46]:
class Stock:

    # Initializing values when class is created
    def __init__(self, ticker, start, end):


        self.ticker = ticker
        self.start = start
        self.end = end
        self.dateRange = pd.date_range(start=self.start, end=self.end, freq=pd.offsets.Week(1))
        self.makeDataDict()
        self.makeDataFrame()
        self.makeColumns()

    # Takes the history of the ticker and puts in dictionary
    def makeDataDict(self):
        self.companies = {}
        self.companies[self.ticker] = yf.Ticker(self.ticker).history(start = self.start, end = self.end)['Close']

    # Creates a DataFrame for every day 1 of the month
    def makeDataFrame(self):
        self.companiesDF = pd.DataFrame(self.companies)
        self.companiesDF = self.companiesDF.groupby(pd.DatetimeIndex(self.companiesDF.index).to_period('W')).nth(0)

    def makeColumns(self):
        self.columns = []
        for column in self.companiesDF.iteritems():
            self.columns.append(column[0])

    # Graphs the price of the Stocks
    def graphStocks(self):
        for x in self.columns:
            plt.plot(self.dateRange, self.companiesDF[x], label = x)
        plt.title("Portfolio Stock Prices", fontsize=20)
        self.makeGraph(True)

    # Adds the Axis and Customization to Graphs
    def makeGraph(self, legend=False):
        plt.xticks(rotation=70)
        plt.xlabel("Dates", fontsize=14)
        plt.ylabel("Price $(USD)", fontsize=14)
        if legend:
            plt.legend(bbox_to_anchor=(1.0, 1.0))

In [47]:
hello = Stock("AAPL", start_date, end_date)

In [48]:
def makeObjects(tickerList):
    objectList = []
    for ticker in tickerList:
        objectList.append(Stock(ticker, start_date, end_date))
    return objectList
    
makeObjects(ticker_list)

In [ ]:
# currently not working with class objects
def findHighestBeta(tickerList):
    '''
    purpose:
        calculates beta between each individual stock and the overall market (SP500 index)
    input:
        tickerList (listof Stock): a list of Stock objects
    output: 
        A list of the beta of each stock in tickerList (assume US market)
    '''
    beta = dict() # to store all beta values

    # calculating market value
    marketIndex='^GSPC'
    marketTicker=yf.Ticker(marketIndex)
    marketHist=marketTicker.history(start=start_date, end=end_date, interval="1mo")
    marketVar=marketHist["Close"].pct_change().var()
    
    # calculating each Beta, then store in the beta dict
    for ticker in tickerList:
        
        #delete later, just access object attributes (make sure it's PERCENT CHANGE)
        tempTicker=yf.Ticker(ticker)
        tempHist=tempTicker.history(start=start_date, end=end_date, interval="1mo")
        temp=pd.DataFrame()
        temp["ticker_close"]=tempHist["Close"].pct_change()
        temp['market_close']=marketHist["Close"].pct_change()
        temp.dropna(inplace=True)

        tempBeta=temp.cov()/marketVar
        #make sure to access the name attribute later
        tempBeta=tempBeta.iloc[0,1]
        beta[ticker]=np.absolute(tempBeta)
        maxBeta = max(beta)
        maxBetaValue = beta.get(maxBeta)
    return maxBeta, maxBetaValue

tickerName, betaValue = findHighestBeta(["AAPL", "SHOP", "NFLX", "GME"])


'SHOP'

In [ ]:
def calculateCorrelation(ticker1, ticker2):
    '''
    purpose:
        calculates the correlation between two stock
    input:
        ticker1 (Stock): Stock object 1
        ticker2 (Stock): Stock object 2
    output: 
        A Dictionary with key of format "ticker1+ticker2" and the 
        value is the correlation between the 2 tickers
    '''
    # change to accessing object attribute
    tickerOne=yf.Ticker(ticker1)
    tickerOneHist=tickerOne.history(start=start_date, end=end_date, interval="1mo")
    tickerTwo=yf.Ticker(ticker2)
    tickerTwoHist=tickerTwo.history(start=start_date, end=end_date, interval="1mo")

    correlation=pd.DataFrame()
    correlation[ticker1]=tickerOneHist["Close"].pct_change()
    correlation[ticker2]=tickerTwoHist["Close"].pct_change()
    correlation.dropna(inplace=True)
    correlation=correlation.corr()
    correlation=correlation.iloc[0,1]

    return {ticker2: correlation}

calculateCorrelation("AAPL", "NFLX")

{'NFLX': 0.38605796156135985}

In [ ]:
def selectPortfolio(maxBetaTicker, tickers):
    portfolio = []
    correlation = []
    for ticker in tickers:
        correlation += calculateCorrelation(maxBetaTicker, ticker)
    
    threshold = 0.9  
    while len(portfolio) <= 11:
        for ticker in correlation:
            if list(ticker.values())[0] >= threshold:
                portfolio += list(ticker.keys())[0]
        if len(portfolio) <= 11:
            portfolio = []
            threshold -= 0.1
    portfolio += maxBetaTicker
    
    return portfolio

## Contribution Declaration

The following team members made a meaningful contribution to this assignment:

- Robin:
- Jessica:
- Eric: 